# Chapter 4 - Discussion Questions 

### [Link](https://runestone.academy/runestone/books/published/pythonds3/BasicDS/DiscussionQuestions.html) 

### 1. Convert the following values to binary using “divide by 2.” Show the stack of remainders.

* 17
* 45
* 96

We can run an interactive Python function to achieve this: it will return the binary string together with the intermediate results.

In [3]:
def divide_by_two(n):
    res = []
    remainders = []

    while n > 0:
        res.append(n // 2)
        remainders.append(str(n % 2))
        n //= 2

    return "".join(remainders), res

Now let's run it on the numbers given in our exercise.

In [4]:
for n in [17, 45, 96]:
    b, r = divide_by_two(n)
    print(f"Binary number for {n} is {b} with stack of remainders {r}.")

Binary number for 17 is 10001 with stack of remainders [8, 4, 2, 1, 0].
Binary number for 45 is 101101 with stack of remainders [22, 11, 5, 2, 1, 0].
Binary number for 96 is 0000011 with stack of remainders [48, 24, 12, 6, 3, 1, 0].


### 2. Convert the following infix expressions to prefix (use full parentheses):

* (A+B)\*(C+D)\*(E+F)
* A+((B+C)\*(D+E))
* A\*B\*C\*D+E+F

This can easily be solved by visual inspection.

* \* \* \* + A B + C D + E F
* \+ A \* + B C + D E
* \+ \+ \* \* \* A B C D E F

We can test this with the appropriate script.

In [5]:
from pythonds3.basic import Stack

def infix_to_prefix(s):
    """ 
    Converts an infix string into the respective prefix representation.
    """
    tokens = list(s)
    operands = Stack()
    operators = Stack()
    
    infix_repr = []

    precedence = {"+": 1, "-": 1, "*": 2, "/": 2, "(": 3}

    for token in tokens:
        if token == "(":
            operands.push(token)
        elif token == ")":
            new_token = str(operators.pop()) + " " + str(operators.pop())
            while operands.peek() != "(":
                new_token = operands.pop() + " " + new_token
            operands.pop()
            operators.push(new_token)
        elif token in precedence.keys():
            lower_prec = []
            while (not operands.is_empty()) and operands.peek() != "(" and precedence[operands.peek()] > precedence[token]:
                lower_prec.append(operands.pop())
            operands.push(token)
            for tok in lower_prec:
                operands.push(tok)
        else:
            operators.push(token)

    while not operators.is_empty():
        infix_repr += [operators.pop()]
    
    infix_repr = infix_repr[::-1]

    while not operands.is_empty():
        infix_repr = [operands.pop()] + infix_repr

    return " ".join(infix_repr)


Let's verify the results.

In [6]:
print(infix_to_prefix("(A+B)*(C+D)*(E+F)"))
print(infix_to_prefix("A+((B+C)*(D+E))"))
print(infix_to_prefix("A*B*C*D+E+F"))

* * + B A + D C + F E
+ A * + E D + C B
+ + * * * A B C D E F


Results are consistent with what we initially computed.

### 3. Convert the above infix expressions to postfix (use full parentheses).

To do this, we can convert the expressions by hand:
* (A B \+) (C D \+) \* (E F \+) \*
* A ((B C \+) (D E \+) \*) \+
* (A B C D \* \* \*) E \+ F \+

### 4. Convert the above infix expressions to postfix using the direct conversion algorithm. Show the stack as the conversion takes place.

We can get the conversion algorithm from the chapter, and slightly modify it for our purposes.

In [22]:
def infix_to_postfix(infix_expr):
    prec = {}
    prec["*"] = 3
    prec["/"] = 3
    prec["+"] = 2
    prec["-"] = 2
    prec["("] = 1
    op_stack = Stack()
    postfix_list = []
    token_list = list(infix_expr)

    for token in token_list:
        print(f"Reading token {token}, current operations stack {op_stack._items}.")
        if token in "ABCDEFGHIJKLMNOPQRSTUVWXYZ" or token in "0123456789":
            postfix_list.append(token)
        elif token == "(":
            op_stack.push(token)
        elif token == ")":
            top_token = op_stack.pop()
            while top_token != "(":
                postfix_list.append(top_token)
                top_token = op_stack.pop()
        else:
            while (not op_stack.is_empty()) and (prec[op_stack.peek()] >= prec[token]):
                postfix_list.append(op_stack.pop())
            op_stack.push(token)

    while not op_stack.is_empty():
        print(f"Emptying stack. Current operands to pop out {op_stack._items}.")
        postfix_list.append(op_stack.pop())

    return " ".join(postfix_list)
    

Now let's run if with the three infix strings.

In [25]:
print(infix_to_postfix("(A+B)*(C+D)*(E+F)"))
print("\n")
print(infix_to_postfix("A+((B+C)*(D+E))"))
print("\n")
print(infix_to_postfix("A*B*C*D+E+F"))

Reading token (, current operations stack [].
Reading token A, current operations stack ['('].
Reading token +, current operations stack ['('].
Reading token B, current operations stack ['(', '+'].
Reading token ), current operations stack ['(', '+'].
Reading token *, current operations stack [].
Reading token (, current operations stack ['*'].
Reading token C, current operations stack ['*', '('].
Reading token +, current operations stack ['*', '('].
Reading token D, current operations stack ['*', '(', '+'].
Reading token ), current operations stack ['*', '(', '+'].
Reading token *, current operations stack ['*'].
Reading token (, current operations stack ['*'].
Reading token E, current operations stack ['*', '('].
Reading token +, current operations stack ['*', '('].
Reading token F, current operations stack ['*', '(', '+'].
Reading token ), current operations stack ['*', '(', '+'].
Emptying stack. Current operands to pop out ['*'].
A B + C D + * E F + *


Reading token A, current ope

It seems that it works!

### 5. Evaluate the following postfix expressions. Show the stack as each operand and operator is processed.
* `2 3 * 4 +`
* `1 2 + 3 + 4 + 5 +`
* `1 2 3 4 5 * + * +`

Again, let's get the code from the chapter and adapt it for our purposes.

In [26]:
def postfix_eval(postfix_expr):
    operand_stack = Stack()
    token_list = postfix_expr.split()

    for token in token_list:
        print(f"Reading token {token}. Current operations stack {operand_stack._items}.")
        if token in "0123456789":
            operand_stack.push(int(token))
        else:
            operand2 = operand_stack.pop()
            operand1 = operand_stack.pop()
            print(f"Running operation {token} on {operand1} and {operand2}.")
            result = do_math(token, operand1, operand2)
            operand_stack.push(result)
    
    return operand_stack.pop()


def do_math(op, op1, op2):
    if op == "*":
        return op1 * op2
    elif op == "/":
        return op1 / op2
    elif op == "+":
        return op1 + op2
    else:
        return op1 - op2


And now let's run the function with our script.

In [27]:
print(postfix_eval("2 3 * 4 +"))
print("\n")
print(postfix_eval("1 2 + 3 + 4 + 5 +"))
print("\n")
print(postfix_eval("1 2 3 4 5 * + * +"))

Reading token 2. Current operations stack [].
Reading token 3. Current operations stack [2].
Reading token *. Current operations stack [2, 3].
Running operation * on 2 and 3.
Reading token 4. Current operations stack [6].
Reading token +. Current operations stack [6, 4].
Running operation + on 6 and 4.
10


Reading token 1. Current operations stack [].
Reading token 2. Current operations stack [1].
Reading token +. Current operations stack [1, 2].
Running operation + on 1 and 2.
Reading token 3. Current operations stack [3].
Reading token +. Current operations stack [3, 3].
Running operation + on 3 and 3.
Reading token 4. Current operations stack [6].
Reading token +. Current operations stack [6, 4].
Running operation + on 6 and 4.
Reading token 5. Current operations stack [10].
Reading token +. Current operations stack [10, 5].
Running operation + on 10 and 5.
15


Reading token 1. Current operations stack [].
Reading token 2. Current operations stack [1].
Reading token 3. Current ope

### 6. The alternative implementation of the Queue ADT is to use a list such that the rear of the queue is at the end of the list. What would this mean for Big-O performance?

Knowing the Big-O performances of the `list` operations `append()` and `insert()`, the operation `enqueue()` would be $O(1)$ while `dequeue()` would be $O(n)$.

### 7. What is the result of carrying out both steps of the linked list `add` method in reverse order? What kind of reference results? What types of problems may result?

Reversing the steps would simply assign `head` to the new node before the new node itself points to the head. In this way we will lose the whole linked list having lost the reference to the first element.

### 8. Explain how the linked list `remove` method works when the item to be removed is in the last node.

The last element's `next` is set to `None`. 

### 9. Explain how the `remove` method works when the item is in the only node in the linked list.

In this case, the `head` is set to `None`.